In [6]:
import pandas as pd
import os
import re
import csv
from Bio import SeqIO

In [15]:
def createMatrix(INDIR, OUTDIR, sample, REF):
    
    infile=INDIR+sample+"_umiConsensus_"+str(numUC)+"_"+str(mutFreq)+"_"+str(quality_threshold)+".tsv" ## input mutation table
    ref = open(REF,"r").readlines()[1]
    out_char=OUTDIR+sample+"_ccs"+".fa"   ## output character matrix
    out_bi=OUTDIR+sample+"_binary"+".tsv"               ## output binary file
    out_phy=OUTDIR+sample+"_temp.phy"                   ## output .phy file for iqtree reconstruction
    with open(infile,"r") as f_in, open(out_bi, "w") as f_d, open(out_phy, "w") as f_d2, open (out_char, "w") as f_D:
        f_in_lines = f_in.readlines()
        numCell = len(f_in_lines)+1
#         f_d2.write(str(numCell)+" "+str(3004)+"\n")
        f_d2.write(str("ref")+" "+''.join(map(str,[0]*3004))+"\n")
        f_D.write(">"+str("ref")+"\n"+ref+"\n")
        for line in f_in_lines:
            muts=line.replace("\n","").split("\t")
            nam="C"+muts[0]
            muts.pop(0)
            muts.pop(0)
            muts = muts[0].split(",")
            types=[re.findall(r'[A-Z]+',s) for s in muts]
            l = len(types)
            subs = []
            #### deletions ############
            D = []
            for i in range(l):
                if types[i]==["D"]:
                    D.append(muts[i])
            D_long = parseDel(D,maxLen=200)
            #### insertions ###########
            I = []
            for i in range(l):
                if types[i]==["I"]:
                    I.append(muts[i])

            ### substitutions ########

            if D_long is None or "LongDels" not in D_long:
                subs = [ele for ele in muts if ele not in D and ele not in I]
                if len(subs)>0:
                    matrixALL = parseSubs(subs, ref)
                    matrix_binary = matrixALL[0]
                    matrix_character = matrixALL[1]
                    
                    f_d2.write(nam+" "+matrix_binary+"\n")
                    writer1=csv.writer(f_d,delimiter=",")
                    writer1.writerow(matrix_binary)
                    f_D.write(">"+nam+"\n"+matrix_character+"\n")

def parseDel(D,maxLen=200):
    for s in D:
        pos = re.findall("(\d+)",s)
        if len(pos) ==2:
            if int(pos[1])-int(pos[0]) > maxLen:
                return("LongDels")
def parseSubs(subs, ref):
    pos=[re.findall(r'[0-9]+',s)[0] for s in subs]
    types=[re.findall(r'[A-Z]+',s)[0] for s in subs]
    pos=[re.findall(r'[0-9]+',s)[0] for s in subs]
    types=[re.findall(r'[A-Z]+',s)[0] for s in subs]
    pos_m=[]
    for i in pos:
        pos_m.append(int(i))
    type_m=[]
    for i in types:
        type_m.append(i)
    s1=[0]*3004
    s2=ref
    for i in range(len(pos_m)):
        pointer = pos_m[i] #-1
        s1[pointer]=1
        s2 = s2[:pointer] + type_m[i][1] + s2[pointer + 1:]
    s0=''.join(map(str,s1))
    return(s0, s2)   ## s0: binary; s2:character sequence


In [16]:
if __name__ == "__main__":
    """
    generate consensus matrix
    """
        
    SAMPLE_BARCODES = "/data/zhaolian/LineageTracing/DSS/PacBio/CCS5passes/samples_all.txt"
    INDIR="/data/zhaolian/LineageTracing/DSS/PacBio/CCS5passes/6.umiConsensus/"
    OUTDIR="/data/zhaolian/LineageTracing/DSS/PacBio/CCS5passes/7.matrix/"
    REF="/data/zhaolian/LineageTracing/DSS/PacBio/reference/3k_HMFonly.fasta"
    numUC=3 ## number of ccs reads in one consensus cluster
    mutFreq=0.6 ## export the mutation if mutation frequency >= mutFreq 
    quality_threshold = 50 ## mapping quality

    if not os.path.exists(OUTDIR):
        os.mkdir(OUTDIR)

    with open(SAMPLE_BARCODES,"r") as f_barcode:
        for line in f_barcode.readlines():
            sample = line.replace("\n", "").split("\t")[0]
            print(sample)
            createMatrix(INDIR, OUTDIR, sample,REF)
            
            #
            out_phy=OUTDIR+sample+"_temp.phy"
            phy = pd.read_csv(out_phy,sep = " ",header = None,names=["num","3004"])
            numR = phy.shape[0]
            phy = phy.rename({'num': str(numR)}, axis='columns')
            phy.to_csv(OUTDIR+sample+".phy",sep = " ",index=False)
            os.remove(out_phy) 



2_1T
2_3T
2N
49_1T
49_2T
49_3T
49N
65_1T
65_3T
65N
4N
5N
16N
47_1N
47_4N
47_5N
47_6N
47_8N
50N
66_1N
4T
5T
16T
47_1T
47_4T
47_5T
47_6T
47_8T
50T
66_1T
